# Bokeh


### basics and some advanced features
<br>
<br>
<br>
<br>




<div align="right">
  Dmitri Stepanov
    <br>
  dmitri.stepanov1@gmail.com
    <br>
    <div>
GitHub @RhDm
        <br>
        </div>
    <div>
Twitter @rh_karu
        </div>
  April 2019
</div>





# Basic example

In [1]:
import numpy as np
from scipy.stats import gamma

from bokeh.plotting import figure
from bokeh.io import show, output_notebook
output_notebook()

# generate some data
alpha=3
x = np.linspace(gamma.ppf(0.01, alpha), gamma.ppf(0.99, alpha), 100)
y= gamma.pdf(x, alpha)

# Bokeh visualisation
p = figure(plot_width=500, plot_height=500, title="Gamma distribution")
p.line(x=x, y=y, line_width=2)
p.xaxis.axis_label = 'Random variable'
p.yaxis.axis_label ='Probability density'

show(p)

Loading BokehJS ...

# Adding hover & styling

In [2]:
from bokeh.models import ColumnDataSource

# for offline usage
# from bokeh.resources import INLINE 
# output_notebook(resources=INLINE)


alpha=3
x = np.linspace(gamma.ppf(0.01, alpha), gamma.ppf(0.99, alpha), 100)
y= gamma.pdf(x, alpha)


source_1 = ColumnDataSource(data={
    'x_1': x,
    'y_1': y})

# tooltips_formatting =[('random variable', '@x_1'), ('PD', '@y_1')]
# or
tooltips_formatting ="""
                    <div>
                        <div>
                            <span style="font-size: 14px; color: #3c6978">x: </span>
                            <span style="font-size: 14px; float: right;  margin: 0px 0px 0px 15px">@x_1</span>
                        </div>
                        <div>
                            <span style="font-size: 14px; color: #3c6978">y:</span>
                            <span style="font-size: 14px; float: right; margin: 0px 0px 0px 15px">@y_1</span>
                        </div>
                    </div>
                    """
    
p = figure(plot_width=700, plot_height=600, title="Gamma distribution", tooltips= tooltips_formatting)

p.line(x='x_1', y='y_1', line_width=2, source= source_1)

p.xaxis.axis_label = 'Gamm random variable'
p.yaxis.axis_label ='Probability density'

p.background_fill_color = "black"
p.background_fill_alpha = 0.99

p.output_backend = "svg"

show(p)

# Plotting many data points

In [3]:
def data_no_noise(x, a, b, c):
    return a * np.exp(-b * x) + c

def generate_data_no_noise(datapoints):
    x = np.linspace(0, 50, datapoints)
    y = data_no_noise(x, 2.5, 1.3, 0.5)
    return x, y  

def generate_data_w_noise(datapoints):
    x, y = generate_data_no_noise(datapoints)
    y_noise = 0.2 * np.random.normal(size=x.size)
    y_with_noise = y + y_noise
    return x, y_with_noise

x, y= generate_data_w_noise(10000)
p = figure(plot_width=600, plot_height=600, x_axis_label='x', y_axis_label='y',
               tools="pan,wheel_zoom,reset,save", x_range=(-1,51))

source = ColumnDataSource(data={
        'x_1': x,
        'y_2': y})
    
    
p.circle(x='x_1', y='y_2', legend="", fill_color="#1f78b4", line_color="#1f78b4", size=6,
             fill_alpha=0.2, source=source)

show(p)

# Adding interactions

In [4]:
from ipywidgets import interactive, interact
import ipywidgets as widgets

def f(a):
    alpha=a
    x = np.linspace(0, gamma.ppf(0.999, alpha), 100)
    y= gamma.pdf(x, alpha)
    p = figure(plot_width=700, plot_height=600, title="Gamma distribution")
    p.line(x=x, y=y, line_width=2)
    p.xaxis.axis_label = 'Gamm random variable'
    p.yaxis.axis_label ='Probability density'
    show(p)

# slider for the shape
interactive_plot = interactive(f, a=widgets.IntSlider(min=1,max=30,step=1,value=3))
interactive_plot

# not good..

interactive(children=(IntSlider(value=3, description='a', max=30, min=1), Output()), _dom_classes=('widget-int…

# Adding interactions

In [5]:
from bokeh.models.widgets import Button, Slider
from bokeh.themes import Theme
from bokeh.layouts import column, row
import yaml

def gamma_shape(doc):
    
    alpha=3
    x = np.linspace(0, gamma.ppf(0.999, alpha), 100)
    y= gamma.pdf(x, alpha)
    
    source = ColumnDataSource(data={
        'x_1': x,
        'y_1': y})

    # tooltips_formatting =[('random variable', '@x_1'), ('PD', '@y_1')]
    # or
    tooltips_formatting ="""
                        <div>
                            <div>
                                <span style="font-size: 14px; color: #3c6978">random variable: </span>
                                <span style="font-size: 14px; float: right;  margin: 0px 0px 0px 15px">@x_1</span>
                            </div>
                            <div>
                                <span style="font-size: 14px; color: #3c6978">max:</span>
                                <span style="font-size: 14px; float: right; margin: 0px 0px 0px 15px">@y_1</span>
                            </div>
                        </div>
                        """

    p = figure(plot_width=700, plot_height=600, title="Gamma distribution", tooltips= tooltips_formatting)

    p.line(x='x_1', y='y_1', line_width=2, source= source)
    
    p.x_range.start = 0
    p.xaxis.axis_label = 'Gamm random variable'
    p.yaxis.axis_label ='Probability density'

    def callback_shape(attr, old, new):
        x_updated = np.linspace(0, gamma.ppf(0.999, new), 100)
        y_updated= gamma.pdf(source.data['x_1'], new)
        new_source= ColumnDataSource(data={
            'x_1': x_updated,
            'y_1': y_updated})   
        source.data.update(new_source.data)

    slider_shape = Slider(start=1, end=30, value=3, step=1, title="Gamma shape")
    slider_shape.on_change('value', callback_shape)

    doc.add_root(column(slider_shape, p))
    doc.theme = Theme(json=yaml.load("""
            attrs:
                Figure:
                    height: 600
                    width: 800
        """))

In [6]:
show(gamma_shape)

/Users/morgan/Desktop/untitled folder/pydata_bokeh/venv/lib/python3.6/site-packages/ipykernel_launcher.py:56: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.


# Adding interactions

In [7]:
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

import warnings
warnings.filterwarnings('ignore')


def data_no_noise(x, a, b, c):
    return a * np.exp(-b * x) + c

def generate_data_no_noise(datapoints):
    x = np.linspace(0, 50, datapoints)
    y = data_no_noise(x, 2.5, 1.3, 0.5)
    return x, y  

def generate_data_w_noise(datapoints):
    x, y = generate_data_no_noise(datapoints)
    y_noise = 0.2 * np.random.normal(size=x.size)
    y_with_noise = y + y_noise
    return x, y_with_noise

def cerate_poly_model(degree, x, y):
    model = make_pipeline(PolynomialFeatures(degree), Ridge())
    model.fit(x.reshape(len(x),1), y.reshape(len(x),1))
    y_pred_poly = model.predict(x.reshape(len(x),1))
    return y_pred_poly
    
def plot_poly_model(doc):
    #initial data and polynomial
    x, y= generate_data_w_noise(100)
    y_data = cerate_poly_model(3, x, y)    
    p = figure(plot_width=600, plot_height=600, x_axis_label='x', y_axis_label='y',
               tools="pan,wheel_zoom,reset,save", x_range=(-1,51))
    source = ColumnDataSource(data={
        'y_data': y_data.ravel(),
        'x_1': x,
        'y_2': y})
    
    p.circle(x='x_1', y='y_2', legend="", fill_color="#1f78b4", line_color="#1f78b4", size=6,
             fill_alpha=0.2, source=source)
    p.line(x='x_1', y='y_data', legend="", line_color="#ef8a62", line_width=4, source= source)
    
    def callback_fix_y_axis():
        p.y_range.start = 0
        p.y_range.end = 3
            
    def callback_deg(attr, old, new):
        y_data= cerate_poly_model(new, source.data['x_1'], source.data['y_2'])    
        new_source= ColumnDataSource(data={
            'y_data': y_data.ravel(),
            'x_1': source.data['x_1'],
            'y_2': source.data['y_2']})   
        source.data.update(new_source.data)

    def callback_data():
        x, y= generate_data_w_noise(slider_num_datapoints.value)
        y_data = cerate_poly_model(slider_deg.value, source.data['x_1'], source.data['y_2'])  
        source.data= ColumnDataSource(data={
            'y_data': y_data.ravel(),
            'x_1': x,
            'y_2': y}).data
        
    def callback_data_n(attr, old, new):
        x, y= generate_data_w_noise(new)
        y_data = cerate_poly_model(slider_deg.value, source.data['x_1'], source.data['y_2'])  
        source.data= ColumnDataSource(data={
            'y_data': y_data.ravel(),
            'x_1': x,
            'y_2': y}).data
            
    slider_deg = Slider(start=1, end=50, value=3, step=1, title="Polynomial degree")
    slider_deg.on_change('value', callback_deg)
    
    slider_num_datapoints = Slider(start=50, end=1000, value=100, step=10, title="Number of data points")
    slider_num_datapoints.on_change('value', callback_data_n)
    
    button_gen = Button(label="Generate new data", button_type="default")
    button_gen.on_click(callback_data)
    
    button_range = Button(label="Fix axis range", button_type="default")
    button_range.on_click(callback_fix_y_axis)
    
    doc.add_root(column(row(slider_deg, slider_num_datapoints, button_gen), button_range, p))
    doc.theme = Theme(json=yaml.load("""
        attrs:
            Figure:
                height: 600
                width: 800
    """))

In [8]:
show(plot_poly_model)

# Embedding Bokeh in web applications

### Flask example

<div style="font-size: 12pt;">
    
```javascript
        flask_app.py
        app/
            __init__.py
            main/
                views.py
                __init__.py
            plot_1/
                views.py
                __init__.py
            plot_2/
                views.py
                __init__.py
            plot_3/
                views.py
                __init__.py
            static/
                styles.css
            templates/
                base.html
                plot_1.html
                plot_2.html
                plot_3.html
                ...
```

</div>

# Bokeh

- easy to use
- interactive plots
- easy to embed in web applications
- powerful
- has "save" button

### Thank you for your attention
<br>
<br>

<div align="left">
  Code: github.com/RhDm/pydata_bokeh.git
    <br>
    <br>
GitHub @RhDm
        <br>
        <br>
Twitter @rh_karu
        <br>
        <br>
dmitri.stepanov1@gmail.com
</div>